# Video Actor Synchroncy and Causality (VASC)
## RAEng: Measuring Responsive Caregiving Project
### Caspar Addyman, 2020
### https://github.com/infantlab/VASC

# Step 1  Process videos using OpenPose

This script uses [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose) human figure recognition neural network to create labeled wireframes for each figure in each frame of a video. It uses the [OpenPoseDemo](https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/demo_overview.md) executable which must already be downloaded and installed.

Additionally, you need to download the trained neural-network models that OpenPose uses. To do this go to the `models` subdirectory of OpenPose directory, and double-click / run the `models.bat` script.

The `openposedemo` bin/exe file can be run manually from the command line.  

OpenPoseDemo will go through a video frame by frame outputing a JSON file for each frame that contain
s a set of coordinate points and for a wireframe for each

In [8]:
#what python libraries to we need?
import os
import sys
import time
import glob
import json

#### Where is OpenPose?

We need the full path to your openpose directory

In [2]:
# location of openposedemo - THIS WILL BE DIFFERENT ON YOUR COMPUTER
openposepath = "C:\\Users\\cas\\openpose-1.5.0-binaries-win64-gpu-python-flir-3d_recommended\\"

if sys.platform == "win32":
    app = "bin\\OpenPoseDemo.exe"
else:
    app = 'bin\\OpenPoseDemo.bin'

openposeapp = openposepath + app
print(openposeapp)

C:\Users\cas\openpose-1.5.0-binaries-win64-gpu-python-flir-3d_recommended\bin\OpenPoseDemo.exe


#### Where are your videos?

In the next cell you need to specify the folder with your set of video files. So that we process them. These scripts use the following director structure. It expects your videos to be in a subfolder of your project 

```
path\to\project\myvideos
```

and then it creates a folder `out` in the project at the same level as the videos with three subfolders for JSON files, the aggregated timeseries and the analyses

```
path\to\project\out\openpose
path\to\project\out\timeseries
path\to\project\out\analyses
```

In [3]:
# where's the project folder? (with trailing slash)
projectpath = os.getcwd() + "\\..\\lookit\\"

# locations of videos and output
videos_in = projectpath 
videos_out_openpose   = projectpath + "out\\openpose"
videos_out_timeseries = projectpath + "out\\timeseries"
videos_out_analyses   = projectpath + "out\\analyses"

print(videos_in)
print(videos_out_openpose)
print(videos_out_timeseries)
print(videos_out_analyses)

C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\
C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\\out\openpose
C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\\out\timeseries
C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\\out\analyses


In [4]:
#first get list of videos in the inbox
avis = glob.glob(videos_in + "*.avi")
mp4s = glob.glob(videos_in + "*.mp4")

print("avis" , len(avis))
print("mp4s" , len(mp4s))

avis 0
mp4s 10


In [5]:
#For the moment we will manually specify what videos to process. 
#TODO generate a list of force or skip videos to automate things slightly
allvideos = []
allvideos.extend(avis)
allvideos.extend(mp4s)
print(len(allvideos))

10


#### Calling the OpenPose app
To operate OpenPose we pass a set of parameters to the demo executable. For the full list of options see  [OpenPoseDemo](https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/demo_overview.md)

Our main parameters are

```
--video        path\to\video_to_process   #input video
--write_json   path\to\output_directory   #one json file per frame
--write_video  path\to\output_directory   #video with identified figures
--write_images path\to\output_directory   #one image per frame with wireframes
--disable_blending true/false             # wireframes on black background (true) or blended on top of video (false)
 ```

Other useful params
 ```
--frame_first  100    #start from frame 100
--display 0           #don't show the images as they are processed
 ```


In [6]:
params = dict()
params["write_json"] = videos_out_openpose
params["write_images"] = videos_out_openpose  #for the moment dump images in output file - TODO name subfolder
params["disable_blending"] = "true"
params["display"]  = "1"

#### The main openpose loop

Call the openpose app for each of the videos

In [7]:
jupwd =  os.getcwd() + "\\" #keep track of current directory so we can change back to it after processing

optstring = ""
for key in params:
    optstring += " --" + key +  ' "' + params[key] + '"' #need to quote paths 

print(optstring)

processed = []
os.chdir(openposepath)
for vid in allvideos:
    #first we need base name of video for the output file name
    base = os.path.basename(vid)
    vidbasename = os.path.splitext(base)
    video_outname = vidbasename[0] + "_output.avi"
    try:
        print("\n\nStaring openpose processing of " + vid)
        # Log the time
        time_start = time.time()
        video = ' --video "' + vid + '"'
        video_out = ' --write_video "' + videos_out_openpose + '\\' + video_outname + '"'
        opbin = openposeapp + video + video_out + optstring
        print(opbin)
        exitcode = os.system(opbin)
        # Log the time again
        time_end = time.time()
        if (exitcode == 0):
            # Print stats
            processed.append(vidbasename[0])
            print ("Done " + vid)
            print ("It took %d seconds for conversion." % (time_end-time_start))
        else:
            print("OpenPose error. Exit code %d" % exitcode)
    except Exception as e:
        print("Error: ", e)
        pass
    
#change the directory back
os.chdir(jupwd)
    
#now we've finished, write a list of processed videos to a file
with open(videos_out_openpose + '\\processed.json', 'w') as outfile:
    json.dump(processed, outfile)




 --write_json "C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\\out\openpose" --write_images "C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\\out\openpose" --disable_blending "true" --display "1"


Staring openpose processing of C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\lookit.01.mp4
C:\Users\cas\openpose-1.5.0-binaries-win64-gpu-python-flir-3d_recommended\bin\OpenPoseDemo.exe --video "C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\lookit.01.mp4" --write_video "C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\\out\openpose\lookit.01_output.avi" --write_json "C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\\out\openpose" --write_images "C:\Users\cas\OneDrive - Goldsmiths College\

Done C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\lookit.09.mp4
It took 17 seconds for conversion.


Staring openpose processing of C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\lookit.10.mp4
C:\Users\cas\openpose-1.5.0-binaries-win64-gpu-python-flir-3d_recommended\bin\OpenPoseDemo.exe --video "C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\lookit.10.mp4" --write_video "C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\\out\openpose\lookit.10_output.avi" --write_json "C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\\out\openpose" --write_images "C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\\out\openpose" --disable_blending "true" --display "1"
Done C:\Users\cas\OneDrive - Goldsm

NameError: name 'json' is not defined

In [ ]:

def video_to_frames(input_loc, output_loc):
    """Function to extract frames from input video file
    and save them as separate frames in an output directory.
    Args:
        input_loc: Input video file.
        output_loc: Output directory to save the frames.
    Returns:
        None
    """
    try:
        os.mkdir(output_loc)
    except OSError:
        pass
    # Log the time
    time_start = time.time()
    # Start capturing the feed
    cap = cv2.VideoCapture(input_loc)
    # Find the number of frames
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print ("Number of frames: ", video_length)
    count = 0
    print ("Converting video..\n")
    # Start converting the video
    while cap.isOpened():
        # Extract the frame
        ret, frame = cap.read()
        # Write the results back to output location.
        cv2.imwrite(output_loc + "/%#05d.jpg" % (count+1), frame)
        count = count + 1
        # If there are no more frames left
        if (count > (video_length-1)):
            # Log the time again
            time_end = time.time()
            # Release the feed
            cap.release()
            # Print stats
            print ("Done extracting frames.\n%d frames extracted" % count)
            print ("It took %d seconds for conversion." % (time_end-time_start))
            break